In [1]:
import pandas as pd
import numpy as np
import os, re
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
def load_msss_data(msss_tables_dir : str, pattern = r"AD_(.*).xls") -> dict:
    """
    Read all MSSS tables from a directory.

    Args:
        msss_tables_dir (str): Folder to search for MSSS table files from.
        pattern (regexp, optional): Pattern to select MSSS table files and capture the label name. Defaults to r"AD_(.*).xls".

    Returns:
        dict: Dictionary of all MSSS data.
    """
    msss_features = {}

    for file in os.listdir(msss_tables_dir):

        match = re.match(pattern=pattern, string=file)

        if not match: continue

        label_name = match.group(1)

        path = os.path.join(msss_tables_dir, file)

        df = pd.read_excel(path, header=1, index_col="Code")

        msss_features[label_name] = df[["Description", "Aux1", "Aux2", "Filter_Key"]]

    return msss_features

In [2]:
msss_codes = load_msss_data("data/eq/datasets/ergon-ffa-msss-tables")

NameError: name 'load_msss_data' is not defined

In [3]:
EQ_DATA_PATH = "data/eq/datasets/2024-tim-elkins-failures-data/"

EQ_DATA_FILES = [
    "EE Failures FY23-24 (Updated).xlsx",
    "EGX Failures FY23-24 (updated).xlsx"
]

EQ_DATA_PROVIDERS = ["Ergon Energy", "Energex"]

EQ_DATA_FILES = [os.path.join(EQ_DATA_PATH, file) for file in EQ_DATA_FILES]

In [4]:
# Load all the datasets
data = [pd.read_excel(dataset) for dataset in EQ_DATA_FILES]

In [5]:
index = "OUTAGE_ID"

input_features = [
"WEATHER_CONDITION",
"OUTAGE_CAUSE",
"FAULT_LONG_DESCRIPTION",
"SHORT_DESC_2",
"WORK_ORDER_COMPONENT_CODE_DESCRIPTION",
"OUTAGE_CAUSE_GROUP",
"OUTAGE_STANDARD_REASON_DESCRIPTION",
"REASON_FOR_INTERRUPTION",
"PROVIDER" # We add this feature to the datasets during pre-processing
]

output_labels = [
    "MSSS_OBJECT_DESCRIPTION",
    "MSSS_DAMAGE_DESCRIPTION",
    "MSSS_CAUSE_DESCRIPTION"
]

In [6]:
# Data backup
# Executing this cell will restore the dataset to the original state

from copy import copy
if "data2" in locals():
    data = data2
data2 = copy(data)

In [7]:
# Capitalize all column names and replace spaces with underscores
for i, dataset in enumerate(data):
    columns = dataset.columns.to_list()
    new_columns = [column.replace(" ", "_").upper() for column in columns]
    replacement = dict(zip(columns, new_columns))
    data[i] = dataset.rename(columns=replacement)

for i, dataset in enumerate(data):
    # Drop all rows with duplicate indices
    # dataset = dataset.drop_duplicates(subset=[index])

    # Set OUTAGE_ID as the index field
    # dataset = dataset.set_index(index)

    # Drop all non input/output columns
    features = input_features + output_labels + [index]
    features = [f for f in features if f in dataset.columns]
    dataset = dataset[features]
    
    # Drop all rows with entirely null input values
    features = [f for f in features if f in input_features]
    dataset = dataset.dropna(subset=features, how='all')
    
    # Add a feature to tell which dataset we're using (EE/EGX)
    provider = EQ_DATA_PROVIDERS[i]
    dataset.insert(dataset.columns.size, "PROVIDER", provider)

    data[i] = dataset

In [15]:
def combine(rows : pd.Series) -> object:
    """
    Combine a series of rows from a given column into one item.

    The combination uses different methods based on the data type of the rows:
    - ``str``: Returns the longest string.
    - ``int``: Returns the mode.
    - ``nan``: Returns ``nan``.

    Args:
        rows (pd.Series): The series of rows.

    Returns:
        object: The combination of the rows.
    """
    # If the series is entirely empty, return NaN
    if rows.value_counts().empty: return np.nan

    # If the series is made of strings:
    if pd.api.types.is_string_dtype(rows):#rows.dtype == "object" and type(rows[]):
        # Return the longest string in the series
        return rows.loc[rows.str.len().idxmax()]
    else:
        # Otherwise, return the mode
        return rows.value_counts().index[0]

In [ ]:
# Combine all rows which have duplicate outage IDs
for i, dataset in enumerate(data):
    dataset = dataset.groupby(index).agg(combine).reset_index()

    data[i] = dataset

In [9]:
# Data backup
# Executing this cell will restore the dataset to the original state

from copy import copy
if "data3" in locals():
    data = data3
data3 = copy(data)

In [97]:
def remove_inconsistent(rows : pd.Series, output_labels : list) -> object:
    if len(rows) == 1: return rows
    if rows.nunique() == 0: return np.nan
    if rows.name not in output_labels:
        return rows.iloc[0]

    if rows.nunique() == 1:
        return rows.iloc[0]
    else:
        print(rows.name)
        print(rows.unique())
        print(f"Inconsistent rows: {rows.index.to_list()}")
        return np.nan

In [111]:
# Remove all inconsistent rows (rows with duplicate input features that have different output labels)
# Remove all rows with duplicate input IDs
for i, dataset in enumerate(data):
    dataset = dataset.groupby(input_features).agg(lambda x : remove_inconsistent(x, output_labels)).reset_index()
    data[i] = dataset

MSSS_OBJECT_DESCRIPTION
['LV Fuse, Unknown' 'Service Fuse - Supply End']
Inconsistent rows: [1753, 1754]


In [ ]:
# dupes = data[1][ data[1].duplicated(keep=False) ]

# # Obtain a 2D list containing all rows in the dataset with duplicate indices grouped by index.
# # Source: https://stackoverflow.com/a/46629549
# duplicate_indices = dupes.groupby(index).apply(lambda x : list(x.index), include_groups=False).tolist()

# for indices in duplicate_indices:

#     # For all rows which have the same Outage ID:
#     rows = data[1].iloc[indices]

#     # Combine all columns in the rows into one.
#     for column in data[1].columns:
        
#         values = rows[column]

#         combined = values.dropna().mode()[0]

#         # Replace the first index with the combined rows
#         first_index = indices[0]
#         #data[1].loc[index, column] = combined
#         data[1][column][first_index] = combined

# data[1] = data[1].drop_duplicates(subset=index,keep='first')
# #data[1] = data[1].drop(rows_to_remove)

In [12]:
# # Drop the index for each dataset
# for i, dataset in enumerate(data):
#     dataset = dataset.reset_index(drop=False)
#     data[i] = dataset

In [289]:
# Concatenate all datasets into one
dataset = pd.concat(data)
dataset = dataset.set_index(index)

In [291]:
new_path = os.path.join(EQ_DATA_PATH, "../preprocessed.csv")

dataset.to_csv(new_path)

new_path = os.path.join(EQ_DATA_PATH, "../preprocessed-supervised.csv")

dataset.dropna(subset=output_labels).to_csv(new_path)

In [81]:
# # Concatenate each sample's input features into strings

# # Get all input features present in the dataset
# inputs = [f for f in input_features if f in dataset.columns]

# # Replace all missing input feature text with "Unknown"
# dataset[inputs] = dataset[inputs].fillna("Unknown")

# # Concatenate each sample's input features into strings
# input_texts = []
# for _, sample in dataset[inputs].iterrows():

#     text = [f"{key}: {value}" for key, value in sample.items()]
#     text = ";\n".join(text)
#     input_texts.append(text)

# # Add all concatenated inputs as a new feature
# dataset["TEXT"] = input_texts

# # Delete individual input features
# dataset = dataset.drop(columns=inputs)

In [103]:
# new_path = os.path.join(EQ_DATA_PATH, "../preprocessed.csv")

# dataset.to_csv(new_path)

In [93]:
import pandas as pd
d = pd.read_csv("../data/eq/datasets/preprocessed.csv", index_col=0)